In [1]:
import numpy as np
from datetime import date

In [2]:
# 이자율, 시행횟수 등 변수지정
n = 10000 ; r = 0.0165
x_vol = 0.249 ; y_vol = 0.2182

In [4]:
# 시간 설정 및 체크포인트
n0 = date.toordinal(date(2018, 6, 29))
n1 = date.toordinal(date(2018, 12, 21))
n2 = date.toordinal(date(2019, 6, 25))
n3 = date.toordinal(date(2019, 12, 23))
n4 = date.toordinal(date(2020, 6, 24))
n5 = date.toordinal(date(2020, 12, 23))
n6 = date.toordinal(date(2021, 6, 24))
check_day = np.array([n1-n0, n2-n0, n3-n0,\
                      n4-n0, n5-n0, n6-n0])
rho = 0.0981; corr = np.array([[1,rho],
                              [rho,1]])
oneyear = 365 ; tot_date = n6 - n0 ; dt = 1/oneyear

In [11]:
# cholesky matrix 만들기
k = np.linalg.cholesky(corr)
S1 = np.zeros((tot_date+1, 1))
S2 = np.zeros((tot_date+1, 1))
# 기초자산의 초기 값
S1[0] = 100; S2[0] = 100
ratio_S1 = S1[0] ; ratio_S2 = S2[0]
strike_price = ([0.90, 0.90, 0.85,\
             0.85, 0.80, 0.75])
# 조기상환 횟수
repay_n = len(strike_price)
# 조기상환시 받는 쿠폰 이자율
coupon_rate = ([0.025, 0.05, 0.075, 0.1, 0.125, 0.15])
# 조기상환시 payoff vector
payment = np.zeros([repay_n, 1])
# payoff vector
payoff = np.zeros([repay_n, 1])
# 전체 payoff vector
tot_payoff = np.zeros([repay_n, 1])
#현가 할인 된 payoff vector
discount_payoff = np.zeros([repay_n, 1])

In [20]:
print(k)
print(w)

[[1.         0.        ]
 [0.0981     0.99517656]]
[[ 0.73220132 -1.0770417   0.62219859 ...  0.4550102   0.59932715
  -1.33305077]
 [-1.28669793 -1.14075812  0.46257505 ... -0.41863921 -1.00410972
  -0.68404501]]


In [12]:
# facevalue, dummy
face_value = 10**4
dummy = 0.15
# knock-in barrier
kib = 0.50

In [19]:
# 조기상환시 페이오프 벡터
for j in range(repay_n):
    payment[j] = face_value*(1+coupon_rate[j])

In [26]:
print(w[0,:])

[ 0.73220132 -1.0770417   0.62219859 ...  0.4550102   0.59932715
 -1.33305077]


In [29]:
# MonteCarlo
for i in range(n):
    # 상관관계가 있는 난수 생성
    w0 = np.random.normal(0, 1, size=[tot_date,2])
    # 난수 행렬 전치
    w0 = np.transpose(w0)
    w = np.matmul(k,w0)
    for j in range(tot_date):
        S1[j+1] = S1[j]*np.exp((r-0.5*x_vol**2)*dt+x_vol*np.sqrt(dt)*w[0,j])
        S2[j+1] = S2[j]*np.exp((r-0.5*y_vol**2)*dt+y_vol*np.sqrt(dt)*w[1,j])
    #[만기평가가격/최초기준가격]의 비율이 더 낮은 가격을 갖는 기초자산
    R1 = S1/ratio_S1 ; R2 = S2/ratio_S2
    WP = np.minimum(R1, R2)
    # 조기상환일 체크하여 페이오프 결정
    WP_checkday = WP[check_day]
    payoff = np.zeros([repay_n,1])
    # 조기 상환이 되지 않은 상태
    repay_event = 0
    for j in range(repay_n):
        if WP_checkday[j] >= strike_price[j]:
            payoff[j] = payment[j] #<- payment:기상환시 페이오프벡터
            #조기상환이 된 상태를 의미
            repay_event = 1
            break
    #조기상환 되지 않고 만기까지 온 경우
    if repay_event == 0:
        if min(WP) > kib:
            #knock in barrier 노터치
            payoff[-1] = face_value*(1+dummy)
        else:
            #knock in barrier 터치
            payoff[-1] = face_value*WP[-1]
            #시뮬레이션마다 페이오프를 더하기
            tot_payoff = tot_payoff + payoff
#모든 시뮬레이션의 페이오프의 평균을 구함
mean_payoff = tot_payoff/n
#페이오프를 무위험 이자율로 할인하여 현재가격을 구함
for j in range(repay_n):
    discount_payoff[j] = mean_payoff[j]\
    *np.exp(-r*check_day[j]/oneyear)
# pricing ELS
price = np.sum(discount_payoff)
print(price)

486.0182052932051
